1\. Write a function that converts number representation (bin<->dec<->hex)

In [ ]:
#the program accept as input three strings defining the two basis and the number to be converted
#it's assumed that the user gives in input a correct representation
#the 0x for hex and the 0b for the binary slices are not compulsory
def conv(f, t, a):
    if f==t:
        return None
    if f=='bin':
        if t == 'hex':
            return hex(int(str(a),2))
        else:
            return int(str(a),2)
    elif f=='dec':
        if t == 'bin':
            return bin(int(a))
        else:
            return hex(int(a))
    else:
        if t == 'dec':
            return int(str(a), 16)
        else:
            return bin(int(str(a),16))
        
def conv2(f, t, a):
    if f == t:
        return None
    if f == 'bin':
        if t == 'hex':
            return conv2('dec','hex',conv2('bin','dec',a))
        else:
            return bintodec(a)
    elif f == 'dec':
        if t == 'hex':
            return dectohex(a)
        else:
            return dectobin(a)
    else:
        if t == 'bin':
            return dectobin(conv2('hex','dec',a))
        else:
            return hextodec(a)

def dectohex(a):
    quo = int(a)
    res = ''
    while quo != 0:
        rem = quo%16
        if rem == 10:
            toadd = 'A'
        elif rem == 11:
            toadd = 'B'
        elif rem == 12:
            toadd = 'C'
        elif rem == 13:
            toadd = 'D'
        elif rem == 14:
            toadd = 'E'
        elif rem == 15:
            toadd = 'F'
        else:
            toadd = str(rem)
        res = toadd + res
        quo = quo // 16
    return "0x"+res

def hextodec(a):
    if a.startswith("0x"):
        a = a[2:]
    
    cont = len(a)-1
    res = 0
    for x in a:
        res = res + int(x)*16**(cont)
        cont -= 1
    return res

def dectobin(a):
    quo = int(a)
    res = ''
    while quo != 0:
        rem = quo%2
        res =  str(rem) + res
        quo = quo // 2
    return "0b"+res

def bintodec(a):    
    if a.startswith("0b"):
        a = a[2:]
    
    cont = len(a)-1
    res = 0
    for x in a:
        res = res + int(x)*2**(cont)
        cont -= 1
    return res

print('dec to hex and dec to bin')
print(conv('dec','hex','341'))
print(conv('dec','bin','341'))
print('second method')
print(conv2('dec','hex','341'))
print(conv2('dec','bin','341'))
print('\n')

print('hex to bin and hex to dec')
print(conv('hex','bin','0x155'))
print(conv('hex','dec','155'))
print('second method')
print(conv('hex','bin','0x155'))
print(conv('hex','dec','155'))
print('\n')

print('bin to hex and bin to dec')
print(conv('bin','hex','101010101'))
print(conv('bin','dec','101010101'))
print('second method')
print(conv2('bin','hex','101010101'))
print(conv2('bin','dec','101010101'))


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [ ]:
def spfp(var):
    var = dectobin(var)
    var = var[2:]
    if (len(var)>32):
        var = var[0:32]
    elif (len(var)<32):
        var = var.zfill(32)
    
    print(var)
    k = "sign: " + str((-1)**int(var[0]))
    k = k+' exponent: (2**'+str((conv('bin','dec',var[1:9])-127))+')'

    cont = 1
    res = 1
    for x in var[9:]:
        res = res + int(x)*2**(-cont)
        cont += 1
    return(k+" mantissa: "+str(res))
    
test = 454546654
spfp(test)

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [ ]:
start = 1.
while 1:
    temp = start/2
    if temp == 0:
        print(start)
        print(temp)
        break
    else:
        start = temp

start = 1.
while 1:
    temp = start*2
    if abs(temp)==float('inf'):
        print(start)
        print(temp)
        break
    else:
        start = temp

4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [ ]:
import numpy as np
start = 1.

step = 2.
end = 0.
while start != end:
    step = step/2
    end = start + step
    
print(step*2)
print(np.finfo(float).eps)

5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

- we have problems of rounding and summing very big/small floats

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [ ]:
from math import sqrt
def std(a,b,c):
    s=sqrt(b**2-4*a*c)
    return ((-b+s)/(2*a),(-b-s)/(2*a))

def std2(a,b,c):
    s=sqrt(b**2-4*a*c)
    return (2*c/(-b-s),2*c/(-b+s))

def accurate(a,b,c):
    return (std2(a,b,c)[0],std(a,b,c)[1])
    
print(std(0.001,1000,0.001))
print(std2(0.001,1000,0.001))
print(accurate(0.001,1000,0.001))

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?
- up to $10^{-8}$ the precision increase, than it starts decreasing, in a bell shape

In [ ]:
func = lambda x: x*(x-1)
x = 1
steps = [10**(-step) for step in range(2,16,2)]
res = []
for delta in steps:
    res.append((func(x+delta)-func(x))/delta)
for x,y in zip(steps, res):
    print("step: %6s res: %5.15f abs_diff: %5.15f" %(x, y, abs(1-y)))

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 

What is requested is on the second version of the function:
- badly we have an high difference
- $1250000$ is the new N to do the run in less than a second, in the first version though N can be increase up to $100000000$
- $10^{-6}$ difference


In [ ]:
from math import sqrt
def integral(start, end, f, N):
    x = np.linspace(start, end, N)
    fx = f(x)
    res = np.sum(fx)*(end-start)/N
    return res

def integral2(start, end, f, N):
    res = 0
    k = np.linspace(start, end, N)
    for i in k:
        res = res + 2/N*f(i)
    return res

func = lambda x: (1-x**2)**0.5
print("N=100")
print("First version")
%time integral(-1,1, func, 100)
res = integral(-1,1, func, 100)
print(res)
print("Second version")
%time integral2(-1,1, func, 100)
print(integral2(-1,1, func, 100))
print("\n")
print("N=1250000")
%time integral(-1,1,func,1250000)
print(integral(-1,1, func, 1250000))
%time integral2(-1,1,func,1250000)
print(integral2(-1,1, func, 1250000))
print("\n")
print("N=100000000")
print("First version")
%time integral(-1,1, func, 100000000)
print(integral(-1,1, func, 100000000))
print("Second version")
%time integral2(-1,1, func, 100000000)
print(integral2(-1,1, func, 100000000))

print("\n")
